<a href="https://colab.research.google.com/github/chow-q/wav2Lip-webUI/blob/main/wav2lip_gfpgan_webui_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/camenduru/Wav2Lip
!git clone https://github.com/TencentARC/GFPGAN.git
!pip install -q gradio==3.50.2 yt_dlp ffmpeg-python librosa==0.8.0
!python /content/GFPGAN/setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P '/content/GFPGAN/experiments/pretrained_models'
%cd Wav2Lip

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72 (delta 0), reused 0 (delta 0), pack-reused 69
Unpacking objects: 100% (72/72), 442.42 KiB | 3.23 MiB/s, done.
Filtering content: 100% (6/6), 936.88 MiB | 60.30 MiB/s, done.
Cloning into 'GFPGAN'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 523 (delta 209), reused 193 (delta 193), pack-reused 272
Receiving objects: 100% (523/523), 5.37 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (280/280), done.
Traceback (most recent call last):
  File "/content/GFPGAN/setup.py", line 83, in <module>
    write_version_py()
  File "/content/GFPGAN/setup.py", line 60, in write_version_py
    with open('VERSION', 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'VERSION'
--2024-01-26 04:07:06--  https://git

In [ ]:
# @title Default title text
!pip install basicsr
!pip install facexlib
!pip install -r ../GFPGANrequirements.txt
!pip install realesrgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.0 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=76c70d0c415160d5c0737fe1c5d5a1f4aab8651183d06e971abe99b31d954cba
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-

In [ ]:
import gradio as gr
import os
import cv2
from tqdm import tqdm
from os import path



def generate(video_in, audio_in):
    import cv2
    import numpy as np
    from tqdm import tqdm
    print(video_in)
    print(audio_in)
    outputPath="/content/Wav2Lip"
    os.system(f"python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '{video_in}' --audio '{audio_in}'")
    #return f"/content/Wav2Lip/results/result_voice.mp4"
    print("把wav2lip生成的视频用fgpgan高清处理")
    #把wav2lip生成的视频用cv2把里面的每一帧画面都转成jpg保存起来
    inputVideoPath = "/content/Wav2Lip/results/result_voice.mp4"
    inputAudioPath = audio_in
    unProcessedFramesFolderPath = '/content/Wav2Lip/results/frames'
    if not os.path.exists(unProcessedFramesFolderPath):
      os.makedirs(unProcessedFramesFolderPath)
    vidcap = cv2.VideoCapture(inputVideoPath)
    numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    print("FPS: ", fps, "Frames: ", numberOfFrames)

    for frameNumber in tqdm(range(numberOfFrames)):
        _,image = vidcap.read()
        cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)
    print("用gfpgan高清修复每一帧画面")
    os.system(f"python /content/GFPGAN/inference_gfpgan.py -i {unProcessedFramesFolderPath} -o {outputPath} -v 1.4 -s 2 --only_center_face --bg_upsampler None")
    #restoredFrames
    restoredFramesPath = outputPath + '/restored_imgs/'
    processedVideoOutputPath = outputPath

    dir_list = os.listdir(restoredFramesPath)
    dir_list.sort()



    batch = 0
    batchSize = 300

    for i in tqdm(range(0, len(dir_list), batchSize)):
      img_array = []
      start, end = i, i+batchSize
      print("processing ", start, end)
      for filename in  tqdm(dir_list[start:end]):
          filename = restoredFramesPath+filename;
          img = cv2.imread(filename)
          if img is None:
            continue
          height, width, layers = img.shape
          size = (width,height)
          img_array.append(img)


      out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
      batch = batch + 1

      for i in range(len(img_array)):
        out.write(img_array[i])
      out.release()
    #合并视频和音频
    concatTextFilePath = outputPath + "/concat.txt"
    concatTextFile=open(concatTextFilePath,"w")
    for ips in range(batch):
      concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
    concatTextFile.close()

    concatedVideoOutputPath = outputPath + "/concated_output.avi"
    os.system(f"!ffmpeg -y -f concat -i {concatTextFilePath} -c copy {concatedVideoOutputPath}")

    finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.avi'
    os.system(f"!ffmpeg -y -i {concatedVideoOutputPath} -i {audio_in} -map 0 -map 1:a -c:v copy -shortest {finalProcessedOuputVideo}")


    return finalProcessedOuputVideo


with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      input_video = gr.Video(label="Input Video")
      input_audio = gr.Audio(label="Input Audio", type='filepath')
      input_generate_button = gr.Button(label="Generate")
    with gr.Column():
        video_out = gr.Video(label="Output Video")
    input_generate_button.click(generate, inputs=[input_video, input_audio], outputs=[video_out])

demo.queue().launch(inline=False, share=True, debug=True)